# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [79]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [80]:
admissions = pd.read_csv("/Users/dramnath/Downloads/CHI-DS-6-master/Unit Projects/project-3/assets/admissions.csv")
admissions = admissions.dropna() 
admissions['prestige'].replace([1.0,2.0,3.0,4.0],['1','2','3','4'],inplace=True)
admissions.head()

,admit,gre,gpa,prestige
0,0,380.0,3.61,3
1,1,660.0,3.67,3
2,1,800.0,4.00,1
3,1,640.0,3.19,4
4,0,520.0,2.93,4


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [81]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(admissions.prestige, admissions.admit, margins=True)

admit,0,1,All
prestige,,,
1,28,33,61
2,95,53,148
3,93,28,121
4,55,12,67
All,271,126,397


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [82]:
dummy_ranks = pd.get_dummies(admissions['prestige'], prefix='prestige')

#### 2.2 When modeling our class variables, how many do we need? 



Answer: 4

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [83]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = admissions[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
handCalc.head()

/Users/dramnath/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


,admit,gre,gpa,prestige_1,prestige_2,prestige_3,prestige_4
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1


In [84]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(handCalc.prestige_1, handCalc.admit, margins=True)

admit,0,1,All
prestige_1,,,
0,243,93,336
1,28,33,61
All,271,126,397


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [26]:
33 / (33 + 28)

0.5409836065573771

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [27]:
28 / (33 + 28)

0.45901639344262296

#### 3.3 Calculate the odds ratio

In [28]:
0.54098 / (1 - .54098)

1.1785543113589823

#### 3.4 Write this finding in a sentenance: 

Answer: The odds of being admitted if your Prestige = 1 is 1.17:1

#### 3.5 Print the cross tab for prestige_4

In [85]:
pd.crosstab(handCalc.prestige_4, handCalc.admit, margins=True)

admit,0,1,All
prestige_4,,,
0,216,114,330
1,55,12,67
All,271,126,397


#### 3.6 Calculate the OR 

In [23]:
0.1791044 / (1 - 0.1791044)

0.21818170300827533

#### 3.7 Write this finding in a sentence

Answer: The odds of being admitted if your Prestige = 4 is .22:1

## Part 4. Analysis

In [86]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = admissions[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
data.head()

,admit,gre,gpa,prestige_2,prestige_3,prestige_4
0,0,380.0,3.61,0,1,0
1,1,660.0,3.67,0,1,0
2,1,800.0,4.00,0,0,0
3,1,640.0,3.19,0,0,1
4,0,520.0,2.93,0,0,1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [87]:
# manually add the intercept
handCalc['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [88]:
train_cols = handCalc.columns[1:]

#### 4.2 Fit the model

In [89]:
logit = sm.Logit(handCalc['admit'], handCalc[train_cols])
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [90]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Tue, 22 Aug 2017   Pseudo R-squ.:                 0.08166
Time:                        20:14:22   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa            0.7793      0.333      2.344      0.019       0.128       1.431
prestige_1    -0.0609   1.77e+07  -3.44e-09      1.000   -3.47e+07    3.47e+07
prestige_2    -0.7411   1.77e+07  -4.18e-08      1.000   -3.47e+07    3.47e+07
prestige_3    -1.3996   1.77e+07   -7.9e-08      1.000   -3.47e+07    3.47e+07
prestige_4    -1.6143   1.77e+07  -9.11e-08      1.000   -3.47e+07    3.47e+07
intercept     -3.8159   1.77e+07  -2.15e-07      1.000   -3.47e+07    3.47e+07
==============================================================================
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [91]:
np.exp(result.params)

gre           1.002221
gpa           2.180027
prestige_1    0.940893
prestige_2    0.476607
prestige_3    0.246695
prestige_4    0.199023
intercept     0.022017
dtype: float64

In [92]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
np.exp(conf)

/Users/dramnath/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """


,2.5%,97.5%,OR
gre,1.000074,1.004372,1.002221
gpa,1.136120,4.183113,2.180027
prestige_1,0.000000,inf,0.940893
prestige_2,0.000000,inf,0.476607
prestige_3,0.000000,inf,0.246695
prestige_4,0.000000,inf,0.199023
intercept,0.000000,inf,0.022017


#### 4.5 Interpret the OR of Prestige_2

Answer: The odds of being admitted multiply by .47 for every prestige value = 2 holding that all other variables are constant. 

#### 4.6 Interpret the OR of GPA

Answer: The odds of being admitted multiply by 2.18 for a 1 unit increase in GPA score.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [93]:
#import cartesian from sklearn
from sklearn.utils.extmath import cartesian

In [94]:
gres = np.linspace(handCalc['gre'].min(), handCalc['gre'].max(), 10)
print (gres)
gpas = np.linspace(handCalc['gpa'].min(), handCalc['gpa'].max(), 10)
print (gpas)

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [95]:
# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]), columns = ['gre', 'gpa', 'prestige', 'intercept'])
combos.head()

,gre,gpa,prestige,intercept
0,220.0,2.260000,1.0,1.0
1,220.0,2.260000,2.0,1.0
2,220.0,2.260000,3.0,1.0
3,220.0,2.260000,4.0,1.0
4,220.0,2.453333,1.0,1.0


#### 5.1 Recreate the dummy variables

In [96]:
# recreate the dummy variables
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy_ranks.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])

/Users/dramnath/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  import sys


#### 5.2 Make predictions on the enumerated dataset

In [99]:
combos['admit_pred'] = result.predict(combos[train_cols])
combos.tail(4)

,gre,gpa,prestige,intercept,prestige_1,prestige_2,prestige_3,prestige_4,admit_pred
396,800.0,4.0,1.0,1.0,1,0,0,0,0.734040
397,800.0,4.0,2.0,1.0,0,1,0,0,0.582995
398,800.0,4.0,3.0,1.0,0,0,1,0,0.419833
399,800.0,4.0,4.0,1.0,0,0,0,1,0.368608


#### 5.3 Interpret findings for the last 4 observations

Answer: With GPA and GRE are constant at the highest possible score, Presitage is the strongest predicitor for acceptance, especially at the higher ranks. A student with a perfect GPA and GRE and Prestige value of 1 has a 73.4% of being addmitted. 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.